In [ ]:
## This was our third attempt at making profit from the stock market and our first attempt at using the pipeline to do so. In order to make more profit and reliably find stocks that were trending upward, we needed to evaluate more than just the SPY. This meant performing computations on each stock in a given universe, which would have been computationally difficult without the pipeline.

## That being said, we were not successful, as this algorithm did not work due to syntax errors in the construction of the pipeline itself and a lack of optimizer in the rebalancing of our portfolio. All together, this was a necessary learning step, as our fourth and final attempt was much more successful.

import quantopian.algorithm as algo
from quantopian.pipeline import Pipeline
from quantopian.algorithm import attach_pipeline, pipeline_output
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.data.psychsignal import stocktwits
from quantopian.pipeline.factors import SimpleMovingAverage
import quantopian.optimize as opt
from quantopian.pipeline.experimental import risk_loading_pipeline

#constant variables to hold the total number of long and short positions allowed
NUM_LONG_POSITIONS = 300
NUM_SHORT_POSTIONS = 300

#initializes, called once
def initialize(context):
    #sets the amount that the brokerage charges for each trade. this is important for backtesting
    #because real trades will cost money to make, so backtesting without using commision will not
    #take into account the money lost on each trade.
    set_commission(commission.PerShare(cost=0.0, min_trade_cost=0))
    #sets the amount that the price will change based on how much is being traded. if a high volume
    #of shares is traded, the price may change before all the orders are filled, which contributes
    #to potential losses.
    set_slippage(slippage.VolumeShareSlippage(volume_limit=1, price_impact=0))
    
    #sets the rebalance function to be called every day at market open
    algo.schedule_function(
        rebalance,
        algo.date_rules.every_day(),
        algo.time_rules.market_open(),
    )

    #sets the record_vars function to be called every day at market close
    algo.schedule_function(
        record_vars,
        algo.date_rules.every_day(),
        algo.time_rules.market_close(),
    )

    #create our dynamic stock selector, also known as 'pipeline', and attach it to this current
    #instance of the user running the algorithm
    algo.attach_pipeline(make_pipeline(), 'pipeline')

#this is where the pipeline is made using all of the perameters and customizations
def make_pipeline():
    
    #the screen will only let chosen stocks through if they belong to this universe
    #in this case, that is the universe of tradable stocks (stocks that exist and are not negative
    #in value) so that we can actually trade the outputs of our pipeline
    universe = QTradableStocksUS() 
    
    #gets 50 day moving average using us equity pricing
    #quantopian will make the computations for us because this is a simple and common computation
    sma_50 = SimpleMovingAverage(
        inputs = [USEquityPricing.close],
        window_length = 50,
    )
    
    #gets 20 day moving average using us equity pricing
    sma_20 = SimpleMovingAverage(
        inputs = [USEquityPricing.close],
        window_length = 20,
    )
     
    #momentum is occuring when the 50 day moving average is greater than the 20 day
    momentum = sma_50 > sma_20
     
    #create a pipeline and add information
    pipe = Pipeline()
    pipe.add(sma_50, 'sma_50')
    pipe.add(sma_20, 'sma_20')
    pipe.set_screen(momentum)

    return pipe

#rebalance will update our portfolio based on what the pipeline outputs and on some simple logic
#that we provided.
#the logic relies on the assumption that when the 50 day sma is greater than the 20 day sma, the
#momentum is upward trending, and when the 20 day sma is greater than the 50 day sma it is not.
def rebalance(context, data):
    #gets the order information, past orders
    open_orders = get_open_orders()
    output = algo.pipeline_output('pipeline')
    
    #if momentum is occuring, buy
    if sma_50 > sma_20:
        if algo not in open_orders:
            order_target_percent(algo, 1.0)
    
    #if momentum not occuring, sell
    elif sma_20 > sma_50:
        if algo not in open_orders:
            order_target_percent(algo, -1.0)